In [1]:
#Imports
import re
import os
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd 
import sklearn
import tensorflow
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, utils, callbacks, optimizers, regularizers
from tensorflow.keras.layers import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Mount and access drive
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive/Few-Shot-Tamil')
print("Change successful.")

Mounted at /content/drive
Change successful.


In [4]:
def tokenize(data):
  generated_token = list(data.split())
  return generated_token

In [5]:
ps = nltk.PorterStemmer()
def stemming(list_of_words):
  stemmed_list = [ps.stem(word) for word in list_of_words]
  return stemmed_list

In [6]:
#Apply TF-IDF
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    train = tfidf_vectorizer.fit_transform(data)
    return train, tfidf_vectorizer

In [7]:
def tamil_preprocessing(dataframe):
  #Remove punctuation symbols
  dataframe['non_punc'] = dataframe['News'].apply(lambda x: re.sub(r'[\'\"\.?!‘]+', '', x.lower()))

  #Tokenize the sentences
  dataframe['tamil_tokenized_text'] = dataframe['non_punc'].apply(lambda x: tokenize(x))

In [8]:
#English translation preprocessing
def eng_preprocessing(dataframe):
  #Remove non-alphanumeric symbols
  dataframe['only_alphanumeric'] = dataframe['English version'].apply(lambda x: re.sub(r'[^a-zA-Z ]+', '', x.lower()))

  #Remove stop-words
  dataframe['imp_text'] = dataframe['only_alphanumeric'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

  #Tokenize the sentences
  dataframe['tokenized_text'] = dataframe['imp_text'].apply(lambda x: tokenize(x))

  #Apply stemming
  dataframe['stemmed_text'] = dataframe['tokenized_text'].apply(lambda x: stemming(x))

In [9]:
#Load the datasets
one_shot_real = pd.read_csv('one-shot-real.csv',  sep='\t')
one_shot_fake = pd.read_csv('one-shot-fake.csv',  sep='\t')
three_shot_real = pd.read_csv('three-shot-real.csv',  sep='\t')
three_shot_fake = pd.read_csv('three-shot-fake.csv',  sep='\t')
five_shot_real = pd.read_csv('five-shot-real.csv',  sep='\t')
five_shot_fake = pd.read_csv('five-shot-fake.csv',  sep='\t')

In [10]:
#Combine the datasets
one_shot = pd.concat([one_shot_real, one_shot_fake])
three_shot = pd.concat([three_shot_real, three_shot_fake])
five_shot = pd.concat([five_shot_real, five_shot_fake])

In [11]:
one_shot['Authenticity'] = [0, 0, 0, 0, 0, 1, 1, 1 ,1, 1]
three_shot['Authenticity'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1]
five_shot['Authenticity'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1]

In [12]:
#Take 10% of the original dataset for testing
os.chdir('/content/drive/My Drive')
news_df = pd.read_csv(filepath_or_buffer='news.csv')
X = news_df['News']
y = news_df['Authenticity']
xtrain, xtest, ytrain, ytest = train_test_split(X, y, stratify=y, random_state=42, test_size=0.1,)

In [13]:
xtest.reset_index(inplace = True, drop = True)

### One-Shot dataset

In [14]:
tamil_preprocessing(one_shot)
eng_preprocessing(one_shot)
one_shot.head()

,English version,Label,News,Author,Date,URL,Authenticity,non_punc,tamil_tokenized_text,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,0,நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]",actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]"
1,A football god about the 'hands of God',['miscellaneous'],‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,0,கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,"[கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]",a football god about the hands of god,football god hands god,"[football, god, hands, god]","[footbal, god, hand, god]"
2,Cameras mandatory in school buses: Tamil Nadu ...,['politics'],பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,செய்திப்பிரிவு,21-Oct-22,https://www.hindutamil.in/news/tamilnadu/88576...,0,பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,"[பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக...",cameras mandatory in school buses tamil nadu g...,cameras mandatory school buses tamil nadu gove...,"[cameras, mandatory, school, buses, tamil, nad...","[camera, mandatori, school, buse, tamil, nadu,..."
3,36th National Games | 4 gold medals for Tamil ...,['sport'],36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,செய்திப்பிரிவு,02-Oct-22,https://www.hindutamil.in/news/sports/877427-3...,0,36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,"[36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம...",th national games gold medals for tamil nadu...,th national games gold medals tamil nadu one day,"[th, national, games, gold, medals, tamil, nad...","[th, nation, game, gold, medal, tamil, nadu, o..."
4,Introducing NoiseFit Core 2 Smartwatch: Amazin...,['tech'],நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,செய்திப்பிரிவு,22-Aug-22,https://www.hindutamil.in/news/technology/8467...,0,நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,"[நாய்ஸ்ஃபிட், கோர், 2, ஸ்மார்ட்வாட்ச், அறிமுகம...",introducing noisefit core smartwatch amazing ...,introducing noisefit core smartwatch amazing f...,"[introducing, noisefit, core, smartwatch, amaz...","[introduc, noisefit, core, smartwatch, amaz, f..."


In [15]:
one_shot.reset_index(inplace = True, drop = True)

In [16]:
one_shot['stemmed_text']

0                         [actor, jayam, ravi, corona]
1                            [footbal, god, hand, god]
2    [camera, mandatori, school, buse, tamil, nadu,...
3    [th, nation, game, gold, medal, tamil, nadu, o...
4    [introduc, noisefit, core, smartwatch, amaz, f...
5                  [say, daniel, radcliff, coronaviru]
6                                   [say, covid, viru]
7    [say, presid, donald, trump, said, peopl, die,...
8    [kieron, pollard, good, viral, kieron, pollard...
9    [pib, fact, check, link, g, radiat, second, wa...
Name: stemmed_text, dtype: object

In [17]:
one_shot['tamil_tokenized_text']

0                             [நடிகர், ஜெயம், ரவிக்கு]
1    [கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]
2    [பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக...
3    [36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம...
4    [நாய்ஸ்ஃபிட், கோர், 2, ஸ்மார்ட்வாட்ச், அறிமுகம...
5    [டேனியல், ராட்கிளிஃப், கொரோனா, வைரஸால், பாதிக்...
6                  [covid-19, வைரஸ், இல்லை, என்கிறார்]
7    [அமெரிக்க, அதிபர், டொனால்ட், டிரம்ப்,, “இதுவரை...
8    [kieron, pollard, நல்லாதான், இருக்காரு:, viral...
9    [pib, fact, check:, 5, ஜி, கதிர்வீச்சுக்கும், ...
Name: tamil_tokenized_text, dtype: object

In [18]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch import nn
from torch.utils.data import DataLoader

In [20]:
#Load a pre-trained model
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model_1 = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=128, activation_function=nn.Tanh())
dense_model_2 = models.Dense(in_features=128, out_features=64, activation_function=nn.Tanh())
dense_model_3 = models.Dense(in_features=64, out_features=32, activation_function=nn.Tanh())

one_shot_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model_1, dense_model_2, dense_model_3])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
#Sentences to train
english_version = one_shot['stemmed_text'].apply(lambda x: ' '.join(x))
tamil_version = one_shot['tamil_tokenized_text'].apply(lambda x: ' '.join(x))

In [22]:
train_examples_one_shot = [
    InputExample(texts=[english_version[0] , tamil_version[0]], label=1.0),
    InputExample(texts=[english_version[1] , tamil_version[1]], label=1.0),
    InputExample(texts=[english_version[2] , tamil_version[2]], label=1.0),
    InputExample(texts=[english_version[3] , tamil_version[3]], label=1.0),
    InputExample(texts=[english_version[4] , tamil_version[4]], label=1.0),
    InputExample(texts=[english_version[5] , tamil_version[5]], label=1.0),    
    InputExample(texts=[english_version[6] , tamil_version[6]], label=1.0),
    InputExample(texts=[english_version[7] , tamil_version[7]], label=1.0),
    InputExample(texts=[english_version[8] , tamil_version[8]], label=1.0),
    InputExample(texts=[english_version[9] , tamil_version[9]], label=1.0),
]


In [23]:
train_dataloader = DataLoader(train_examples_one_shot, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(one_shot_model)

In [24]:
one_shot_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

### Three-Shot dataset

In [25]:
tamil_preprocessing(three_shot)
eng_preprocessing(three_shot)
three_shot.head()

,English version,Label,News,Author,Date,URL,Authenticity,non_punc,tamil_tokenized_text,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,0,நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]",actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]"
1,Pooja Hegde regrets the film's failure,['entertainment'],படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,தினத்தந்தி,22-10-2022 02:50,https://www.dailythanthi.com/Cinema/CinemaNews...,0,படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,"[படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]",pooja hegde regrets the films failure,pooja hegde regrets films failure,"[pooja, hegde, regrets, films, failure]","[pooja, hegd, regret, film, failur]"
2,Images for Diwali,['entertainment'],தீபாவளிக்கு வரும் படங்கள்,தினத்தந்தி,17-10-2022 10:20,https://www.dailythanthi.com/Cinema/CinemaNews...,0,தீபாவளிக்கு வரும் படங்கள்,"[தீபாவளிக்கு, வரும், படங்கள்]",images for diwali,images diwali,"[images, diwali]","[imag, diwali]"
3,A football god about the 'hands of God',['miscellaneous'],‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,0,கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,"[கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]",a football god about the hands of god,football god hands god,"[football, god, hands, god]","[footbal, god, hand, god]"
4,200 km on two jugs of water. Car trip!,['miscellaneous'],இரண்டு குடம் தண்ணீரில் 200 கி.மீ. கார் பயணம்!,அண்டன் பிரகாஷ்,16-Nov-21,https://www.hindutamil.in/news/supplements/ila...,0,இரண்டு குடம் தண்ணீரில் 200 கிமீ கார் பயணம்,"[இரண்டு, குடம், தண்ணீரில், 200, கிமீ, கார், பய...",km on two jugs of water car trip,km two jugs water car trip,"[km, two, jugs, water, car, trip]","[km, two, jug, water, car, trip]"


In [26]:
three_shot.reset_index(inplace = True, drop = True)

In [27]:
three_shot['stemmed_text']

0                          [actor, jayam, ravi, corona]
1                   [pooja, hegd, regret, film, failur]
2                                        [imag, diwali]
3                             [footbal, god, hand, god]
4                      [km, two, jug, water, car, trip]
5                     [internet, world, indian, search]
6     [camera, mandatori, school, buse, tamil, nadu,...
7     [case, translat, central, govern, scheme, name...
8     [russia, plan, start, product, ak, rifl, india...
9     [th, nation, game, gold, medal, tamil, nadu, o...
10           [world, cup, bumrah, yet, ganguli, inform]
11    [kkr, got, suryakumar, yadav, career, breakthr...
12    [introduc, noisefit, core, smartwatch, amaz, f...
13    [googl, releas, android, version, pixel, phone...
14    [whatsapp, introduc, new, privaci, featur, ful...
15                  [say, daniel, radcliff, coronaviru]
16    [say, kobe, bryant, wife, record, suicid, depr...
17    [say, bill, cosbi, frame, sexual, assault,

In [28]:
three_shot['tamil_tokenized_text']

0                              [நடிகர், ஜெயம், ரவிக்கு]
1        [படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]
2                         [தீபாவளிக்கு, வரும், படங்கள்]
3     [கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]
4     [இரண்டு, குடம், தண்ணீரில், 200, கிமீ, கார், பய...
5     [2020, இணைய, உலகம்:, இந்தியர்கள், யாரைத், தேடி...
6     [பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக...
7     [மத்திய, அரசு, திட்ட, பெயர்களை, தமிழில், மொழிப...
8     [இந்தியாவில், ஏகே, 203’, துப்பாக்கிகள், தயாரிப...
9     [36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம...
10    [டி20, உலகக், கோப்பை, |, பும்ரா, இன்னும், வெளி...
11    [சூர்யகுமார், யாதவ், கரியருக்கு, கேகேஆர், அணிய...
12    [நாய்ஸ்ஃபிட், கோர், 2, ஸ்மார்ட்வாட்ச், அறிமுகம...
13    [பிக்சல், போனில், ஆண்ட்ராய்டு, 13, வெர்ஷனை, வெ...
14    [புதிய, பிரைவசி, அம்சங்களை, அறிமுகம், செய்த, வ...
15    [டேனியல், ராட்கிளிஃப், கொரோனா, வைரஸால், பாதிக்...
16    [கோபி, பிரையன்ட்டின், மனைவி, இன்று, காலை, மனச்...
17    [பில், காஸ்பி, என்பிசியை, வாங்குவதைத், தடு

In [29]:
#Load a pre-trained model
three_shot_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model_1, dense_model_2, dense_model_3])

In [30]:
#Sentences to train
english_version_three = three_shot['stemmed_text'].apply(lambda x: ' '.join(x))
tamil_version_three = three_shot['tamil_tokenized_text'].apply(lambda x: ' '.join(x))

In [31]:
train_examples_three_shot = [
    InputExample(texts=[english_version_three[0] , tamil_version_three[0]], label=1.0),
    InputExample(texts=[english_version_three[1] , tamil_version_three[1]], label=1.0),
    InputExample(texts=[english_version_three[2] , tamil_version_three[2]], label=1.0),
    InputExample(texts=[english_version_three[3] , tamil_version_three[3]], label=1.0),
    InputExample(texts=[english_version_three[4] , tamil_version_three[4]], label=1.0),
    InputExample(texts=[english_version_three[5] , tamil_version_three[5]], label=1.0),    
    InputExample(texts=[english_version_three[6] , tamil_version_three[6]], label=1.0),
    InputExample(texts=[english_version_three[7] , tamil_version_three[7]], label=1.0),
    InputExample(texts=[english_version_three[8] , tamil_version_three[8]], label=1.0),
    InputExample(texts=[english_version_three[9] , tamil_version_three[9]], label=1.0),
    InputExample(texts=[english_version_three[10] , tamil_version_three[10]], label=1.0),
    InputExample(texts=[english_version_three[11] , tamil_version_three[11]], label=1.0),
    InputExample(texts=[english_version_three[12] , tamil_version_three[12]], label=1.0),
    InputExample(texts=[english_version_three[13] , tamil_version_three[13]], label=1.0),
    InputExample(texts=[english_version_three[14] , tamil_version_three[14]], label=1.0),
    InputExample(texts=[english_version_three[15] , tamil_version_three[15]], label=1.0),
    InputExample(texts=[english_version_three[16] , tamil_version_three[16]], label=1.0),
    InputExample(texts=[english_version_three[17] , tamil_version_three[17]], label=1.0),
    InputExample(texts=[english_version_three[18] , tamil_version_three[18]], label=1.0),
    InputExample(texts=[english_version_three[19] , tamil_version_three[19]], label=1.0),
    InputExample(texts=[english_version_three[20] , tamil_version_three[20]], label=1.0),
    InputExample(texts=[english_version_three[21] , tamil_version_three[21]], label=1.0),
    InputExample(texts=[english_version_three[22] , tamil_version_three[22]], label=1.0),
    InputExample(texts=[english_version_three[23] , tamil_version_three[23]], label=1.0),
    InputExample(texts=[english_version_three[24] , tamil_version_three[24]], label=1.0),
    InputExample(texts=[english_version_three[25] , tamil_version_three[25]], label=1.0),
    InputExample(texts=[english_version_three[26] , tamil_version_three[26]], label=1.0),
    InputExample(texts=[english_version_three[27] , tamil_version_three[27]], label=1.0),
    InputExample(texts=[english_version_three[28] , tamil_version_three[28]], label=1.0),
    InputExample(texts=[english_version_three[29] , tamil_version_three[29]], label=1.0),
    
]


In [32]:
train_dataloader = DataLoader(train_examples_three_shot, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(three_shot_model)

In [33]:
three_shot_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
embedding = one_shot_model.encode(english_version[0])


In [35]:
print(embedding)

[-0.02248907 -0.15817627 -0.16180572  0.1152757   0.18726152 -0.03689953
  0.03176001  0.03463018  0.01844183 -0.15674624 -0.04031822  0.05702743
 -0.0294059  -0.07465007  0.11593464 -0.11375984 -0.15314358  0.03533853
  0.0901054   0.09368625  0.09502967  0.17636363  0.12502025 -0.01142217
 -0.20560372  0.14510068 -0.18851587  0.02562457 -0.08295032 -0.02386667
  0.07452732 -0.09398662]


### Five-Shot dataset

In [36]:
tamil_preprocessing(five_shot)
eng_preprocessing(five_shot)
five_shot.head()

,English version,Label,News,Author,Date,URL,Authenticity,non_punc,tamil_tokenized_text,only_alphanumeric,imp_text,tokenized_text,stemmed_text
0,Actor Jayam Ravi has Corona,['entertainment'],நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,0,நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]",actor jayam ravi has corona,actor jayam ravi corona,"[actor, jayam, ravi, corona]","[actor, jayam, ravi, corona]"
1,Pooja Hegde regrets the film's failure,['entertainment'],படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,தினத்தந்தி,22-10-2022 02:50,https://www.dailythanthi.com/Cinema/CinemaNews...,0,படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,"[படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]",pooja hegde regrets the films failure,pooja hegde regrets films failure,"[pooja, hegde, regrets, films, failure]","[pooja, hegd, regret, film, failur]"
2,Images for Diwali,['entertainment'],தீபாவளிக்கு வரும் படங்கள்,தினத்தந்தி,17-10-2022 10:20,https://www.dailythanthi.com/Cinema/CinemaNews...,0,தீபாவளிக்கு வரும் படங்கள்,"[தீபாவளிக்கு, வரும், படங்கள்]",images for diwali,images diwali,"[images, diwali]","[imag, diwali]"
3,Simbu debuts in Bollywood,['entertainment'],பாலிவுட்டில் அறிமுகமாகிறார் சிம்பு,தினத்தந்தி,16-10-2022 03:49,https://www.dailythanthi.com/Cinema/CinemaNews...,0,பாலிவுட்டில் அறிமுகமாகிறார் சிம்பு,"[பாலிவுட்டில், அறிமுகமாகிறார், சிம்பு]",simbu debuts in bollywood,simbu debuts bollywood,"[simbu, debuts, bollywood]","[simbu, debut, bollywood]"
4,3rd song release from 'Prince',['entertainment'],'பிரின்ஸ்' படத்தின் 3வது பாடல் வெளியீடு,தினத்தந்தி,14-10-2022 16:31,https://www.dailythanthi.com/Cinema/CinemaNews...,0,பிரின்ஸ் படத்தின் 3வது பாடல் வெளியீடு,"[பிரின்ஸ், படத்தின், 3வது, பாடல், வெளியீடு]",rd song release from prince,rd song release prince,"[rd, song, release, prince]","[rd, song, releas, princ]"


In [37]:
five_shot.reset_index(inplace = True, drop = True)

In [38]:
five_shot['stemmed_text']

0                          [actor, jayam, ravi, corona]
1                   [pooja, hegd, regret, film, failur]
2                                        [imag, diwali]
3                             [simbu, debut, bollywood]
4                             [rd, song, releas, princ]
5                             [footbal, god, hand, god]
6                      [km, two, jug, water, car, trip]
7                     [internet, world, indian, search]
8                                [social, media, junki]
9                      [viral, walk, coatsuit, plastic]
10    [camera, mandatori, school, buse, tamil, nadu,...
11    [case, translat, central, govern, scheme, name...
12    [russia, plan, start, product, ak, rifl, india...
13    [case, polic, offic, kidnap, businessman, buy,...
14    [tamil, hesit, stage, massiv, protest, hindi, ...
15    [th, nation, game, gold, medal, tamil, nadu, o...
16           [world, cup, bumrah, yet, ganguli, inform]
17    [kkr, got, suryakumar, yadav, career, brea

In [39]:
five_shot['tamil_tokenized_text']

0                              [நடிகர், ஜெயம், ரவிக்கு]
1        [படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]
2                         [தீபாவளிக்கு, வரும், படங்கள்]
3                [பாலிவுட்டில், அறிமுகமாகிறார், சிம்பு]
4           [பிரின்ஸ், படத்தின், 3வது, பாடல், வெளியீடு]
5     [கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]
6     [இரண்டு, குடம், தண்ணீரில், 200, கிமீ, கார், பய...
7     [2020, இணைய, உலகம்:, இந்தியர்கள், யாரைத், தேடி...
8       [சமூக, ஊடகத்தில், அதிகம், புழங்குபவரா, நீங்கள்]
9               [வைரல், உலா:, பிளாஸ்டிக்கில், கோட்சூட்]
10    [பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக...
11    [மத்திய, அரசு, திட்ட, பெயர்களை, தமிழில், மொழிப...
12    [இந்தியாவில், ஏகே, 203’, துப்பாக்கிகள், தயாரிப...
13    [தொழிலதிபரை, கடத்தி, சொத்துகளை, எழுதி, வாங்கிய...
14    [இந்தி, திணிப்புக்கு, எதிராக, மிகப்பெரிய, போர...
15    [36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம...
16    [டி20, உலகக், கோப்பை, |, பும்ரா, இன்னும், வெளி...
17    [சூர்யகுமார், யாதவ், கரியருக்கு, கேகேஆர், 

In [40]:
five_shot['tamil_tokenized_text']

0                              [நடிகர், ஜெயம், ரவிக்கு]
1        [படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]
2                         [தீபாவளிக்கு, வரும், படங்கள்]
3                [பாலிவுட்டில், அறிமுகமாகிறார், சிம்பு]
4           [பிரின்ஸ், படத்தின், 3வது, பாடல், வெளியீடு]
5     [கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]
6     [இரண்டு, குடம், தண்ணீரில், 200, கிமீ, கார், பய...
7     [2020, இணைய, உலகம்:, இந்தியர்கள், யாரைத், தேடி...
8       [சமூக, ஊடகத்தில், அதிகம், புழங்குபவரா, நீங்கள்]
9               [வைரல், உலா:, பிளாஸ்டிக்கில், கோட்சூட்]
10    [பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக...
11    [மத்திய, அரசு, திட்ட, பெயர்களை, தமிழில், மொழிப...
12    [இந்தியாவில், ஏகே, 203’, துப்பாக்கிகள், தயாரிப...
13    [தொழிலதிபரை, கடத்தி, சொத்துகளை, எழுதி, வாங்கிய...
14    [இந்தி, திணிப்புக்கு, எதிராக, மிகப்பெரிய, போர...
15    [36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம...
16    [டி20, உலகக், கோப்பை, |, பும்ரா, இன்னும், வெளி...
17    [சூர்யகுமார், யாதவ், கரியருக்கு, கேகேஆர், 

In [41]:
#Load a pre-trained model
five_shot_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model_1, dense_model_2, dense_model_3])

In [42]:
#Sentences to train
english_version_five = five_shot['stemmed_text'].apply(lambda x: ' '.join(x))
tamil_version_five = five_shot['tamil_tokenized_text'].apply(lambda x: ' '.join(x))

In [43]:
train_examples_five_shot = [
    InputExample(texts=[english_version_five[0] , tamil_version_five[0]], label=1.0),
    InputExample(texts=[english_version_five[1] , tamil_version_five[1]], label=1.0),
    InputExample(texts=[english_version_five[2] , tamil_version_five[2]], label=1.0),
    InputExample(texts=[english_version_five[3] , tamil_version_five[3]], label=1.0),
    InputExample(texts=[english_version_five[4] , tamil_version_five[4]], label=1.0),
    InputExample(texts=[english_version_five[5] , tamil_version_five[5]], label=1.0),    
    InputExample(texts=[english_version_five[6] , tamil_version_five[6]], label=1.0),
    InputExample(texts=[english_version_five[7] , tamil_version_five[7]], label=1.0),
    InputExample(texts=[english_version_five[8] , tamil_version_five[8]], label=1.0),
    InputExample(texts=[english_version_five[9] , tamil_version_five[9]], label=1.0),
    InputExample(texts=[english_version_five[10] , tamil_version_five[10]], label=1.0),
    InputExample(texts=[english_version_five[11] , tamil_version_five[11]], label=1.0),
    InputExample(texts=[english_version_five[12] , tamil_version_five[12]], label=1.0),
    InputExample(texts=[english_version_five[13] , tamil_version_five[13]], label=1.0),
    InputExample(texts=[english_version_five[14] , tamil_version_five[14]], label=1.0),
    InputExample(texts=[english_version_five[15] , tamil_version_five[15]], label=1.0),
    InputExample(texts=[english_version_five[16] , tamil_version_five[16]], label=1.0),
    InputExample(texts=[english_version_five[17] , tamil_version_five[17]], label=1.0),
    InputExample(texts=[english_version_five[18] , tamil_version_five[18]], label=1.0),
    InputExample(texts=[english_version_five[19] , tamil_version_five[19]], label=1.0),
    InputExample(texts=[english_version_five[20] , tamil_version_five[20]], label=1.0),
    InputExample(texts=[english_version_five[21] , tamil_version_five[21]], label=1.0),
    InputExample(texts=[english_version_five[22] , tamil_version_five[22]], label=1.0),
    InputExample(texts=[english_version_five[23] , tamil_version_five[23]], label=1.0),
    InputExample(texts=[english_version_five[24] , tamil_version_five[24]], label=1.0),
    InputExample(texts=[english_version_five[25] , tamil_version_five[25]], label=1.0),
    InputExample(texts=[english_version_five[26] , tamil_version_five[26]], label=1.0),
    InputExample(texts=[english_version_five[27] , tamil_version_five[27]], label=1.0),
    InputExample(texts=[english_version_five[28] , tamil_version_five[28]], label=1.0),
    InputExample(texts=[english_version_five[29] , tamil_version_five[29]], label=1.0),
    InputExample(texts=[english_version_five[30] , tamil_version_five[30]], label=1.0),
    InputExample(texts=[english_version_five[31] , tamil_version_five[31]], label=1.0),
    InputExample(texts=[english_version_five[32] , tamil_version_five[32]], label=1.0),
    InputExample(texts=[english_version_five[33] , tamil_version_five[33]], label=1.0),
    InputExample(texts=[english_version_five[34] , tamil_version_five[34]], label=1.0),
    InputExample(texts=[english_version_five[35] , tamil_version_five[35]], label=1.0),
    InputExample(texts=[english_version_five[36] , tamil_version_five[36]], label=1.0),
    InputExample(texts=[english_version_five[37] , tamil_version_five[37]], label=1.0),
    InputExample(texts=[english_version_five[38] , tamil_version_five[38]], label=1.0),
    InputExample(texts=[english_version_five[39] , tamil_version_five[39]], label=1.0),
    InputExample(texts=[english_version_five[40] , tamil_version_five[40]], label=1.0),
    InputExample(texts=[english_version_five[41] , tamil_version_five[41]], label=1.0),
    InputExample(texts=[english_version_five[42] , tamil_version_five[42]], label=1.0),
    InputExample(texts=[english_version_five[43] , tamil_version_five[43]], label=1.0),
    InputExample(texts=[english_version_five[44] , tamil_version_five[44]], label=1.0),
    InputExample(texts=[english_version_five[45] , tamil_version_five[45]], label=1.0),
    InputExample(texts=[english_version_five[46] , tamil_version_five[46]], label=1.0),
    InputExample(texts=[english_version_five[47] , tamil_version_five[47]], label=1.0),
    InputExample(texts=[english_version_five[48] , tamil_version_five[48]], label=1.0),
    InputExample(texts=[english_version_five[49] , tamil_version_five[49]], label=1.0),
    
]


In [44]:
train_dataloader = DataLoader(train_examples_five_shot, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(five_shot_model)

In [45]:
five_shot_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]